<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/infer_w_Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import openai

# It's recommended to use environment variables or Colab secrets for API keys
# For demonstration, keeping it here, but be cautious with sharing notebooks with keys

def run_prompt(prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2000,  # Increased max_tokens to allow for longer response
        temperature=0.7,
        n=1,
        stop=None
    )
    return response.choices[0].message.content

In [8]:
import pandas as pd
import numpy as np
import re
from google.colab import drive
drive.mount('/content/drive')
import plotly.graph_objects as go
import json
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
FILE_PATH = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/columns_[('Image_Search', 'Browser_history', 'Location_history', 'YT_search_history')]_inferenceNo_[3]_combinations_[1, 2, 3, 4]_20251021_005136.json"

In [ ]:
import json
import re
# Assuming run_prompt function is defined and available from a previous cell

# --- Load your JSON data ---
# Ensure FILE_PATH is defined in a previous cell
# FILE_PATH = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/columns_[('Image_Search', 'Browser_history', 'Location_history', 'YT_search_history')]_inferenceNo_[3]_combinations_[1, 2, 3, 4]_20251021_005136.json" # Assuming FILE_PATH is set elsewhere

with open(FILE_PATH, "r") as f:
    data = json.load(f)

# --- Extract all inferences ---
all_inferences = []
for entry in data:
    inferences = entry.get("gpt_output", {}).get("inferences", [])
    for inf in inferences:
        if "inference" in inf:
            all_inferences.append(inf["inference"])

# --- Prepare prompt for thematic labeling ---
# import json # Already imported

In [13]:
import json
import re

def extract_inference_details(inference_text):
    """
    Use GPT to split an inference into category, activity, and reason.
    Falls back to default parsing if model output isn't valid JSON.
    """

    prompt = f"""
    You are given an inference about a user's behavior or interest.
    Extract and return a JSON object with the following keys:
    - category: a short theme (e.g., 'Health & Fitness', 'Travel', 'Cooking')
    - activity: what the user is doing or interested in
    - reason: the evidence or rationale given in the sentence

    Example:
    Input: "The user is interested in health and fitness, as indicated by their search for exercises like 'leg raises' and 'calf raises', as well as their queries about tendonitis and Pilates."
    Output: {{
      "category": "Health & Fitness",
      "activity": "Exercise Routine like Pilates",
      "reason": "User searched for exercises such as leg raises and calf raises, and looked up information on tendonitis and Pilates."
    }}

    Now extract for this input:
    "{inference_text}"
    Return ONLY valid JSON.
    """

    reply = run_prompt(prompt)  # Your existing GPT call
    match = re.search(r"\{.*\}", reply, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass

    # Fallback default structure if parsing fails
    return {
        "category": "Other",
        "activity": inference_text.split(",")[0].replace("The user", "User").strip(),
        "reason": inference_text
    }

def add_inference_details(data):
    """
    Add category, activity, and reason for each inference in the dataset.
    """
    for entry in data:
        gpt_output = entry.get("gpt_output", {})
        inferences = gpt_output.get("inferences", [])

        for inf in inferences:
            details = extract_inference_details(inf["inference"])
            inf.update(details)  # Add new keys to each inference

    return data


In [ ]:
# ---- Run on your input file ----

updated_data = add_inference_details(data)

with open("output_with_details.json", "w") as f:
    json.dump(updated_data, f, indent=2)

# print(f"✅ Done! Added category, activity, and reason to all inferences. Saved to {output_path}")